In [1]:
# Importing packages

import pandas as pd
import numpy  as np
from sklearn.cluster import KMeans
import seaborn as sns
import datetime 
import chardet
import re
from collections import Counter



# tBaustoff Dataset

The tBaustoff dataset provides material mapping information with 10 columns. It connects to the ÖKOBAUDAT dataset through process UUIDs and contains end-of-life scenario information for various materials.

End-of-Life Scenarios: Different materials have different potential pathways

- Reuse: Direct application in new construction (highest value retention)
- Recycling: Processing into new material (partial value retention)
- Downcycling: Processing into lower-value applications
- Disposal: Landfilling or incineration (lowest circularity value)

## Data Wrangling

### Data Extraction

In [2]:
# detect files encoding
with open("/Users/pablosoriano/Documents/Data Science/bbsr-challenge/csv/OBD_2024_I.csv", "rb") as f:
    result = chardet.detect(f.read(100000))  # Read first 100,000 bytes
    print(result)
# Importing data from CSV and converting to dataframe
obd_df = pd.read_csv("/Users/pablosoriano/Documents/Data Science/bbsr-challenge/csv/obd_merged.csv", delimiter=";", encoding = "utf-8-sig", low_memory=False)
tbau_df = pd.read_csv("/Users/pablosoriano/Documents/Data Science/bbsr-challenge/csv/tBaustoff_with_OBD_mapping.csv", delimiter=",", encoding = result["encoding"], low_memory=False)


{'encoding': 'ISO-8859-1', 'confidence': 0.7295400999999999, 'language': ''}


In [3]:
obd_df.shape

(25665, 76)

In [4]:
tbau_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 815 entries, 0 to 814
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   oekobaudatProcessUuid        728 non-null    object 
 1   oekobaudatDatastockUuid      731 non-null    object 
 2   oekobaudatDatastockName      731 non-null    object 
 3   productId                    815 non-null    int64  
 4   tBaustoffVersion             815 non-null    object 
 5   productName                  815 non-null    object 
 6   eolCategoryId                815 non-null    int64  
 7   eolCategoryName              815 non-null    object 
 8   eolScenarioUnbuiltReal       815 non-null    object 
 9   eolScenarioUnbuiltPotential  815 non-null    object 
 10  technologyFactor             815 non-null    float64
dtypes: float64(1), int64(2), object(8)
memory usage: 70.2+ KB


In [5]:
tbau_df.head(6)

,oekobaudatProcessUuid,oekobaudatDatastockUuid,oekobaudatDatastockName,productId,tBaustoffVersion,productName,eolCategoryId,eolCategoryName,eolScenarioUnbuiltReal,eolScenarioUnbuiltPotential,technologyFactor
0,3b1f0c75-07a4-4182-b310-5529fc5b54a6,22885a6e-1765-4ade-a35e-ae668bd07256,OBD_2023_I,1,2024-Q4,Acetyliertes Holz,1,Holz - acetyliert,EV+,EV+,0.00
1,0dec19c7-53db-4cb9-bbba-4d44d3da62a9,ca70a7e6-0ea4-4e90-a947-d44585783626,OBD_2024_I,1,2024-Q4,Acetyliertes Holz,1,Holz - acetyliert,EV+,EV+,0.00
2,bd6d6d89-b76d-4002-a217-afffbb8aa308,448d1096-2017-4901-a560-f652a83c737e,OBD_2020_II,2,2024-Q4,Aluminium Profil,2,Alu unbeschichtet od. eloxiert (Sz. Knetleg zu...,RC+,CL+,0.75
3,fdc99ab8-d843-44ec-a66c-92367d244321,22885a6e-1765-4ade-a35e-ae668bd07256,OBD_2023_I,2,2024-Q4,Aluminium Profil,2,Alu unbeschichtet od. eloxiert (Sz. Knetleg zu...,RC+,CL+,0.75
4,3feca796-791b-46d3-8160-95ef243ffb9d,ca70a7e6-0ea4-4e90-a947-d44585783626,OBD_2024_I,2,2024-Q4,Aluminium Profil,2,Alu unbeschichtet od. eloxiert (Sz. Knetleg zu...,RC+,CL+,0.75
5,a4c1c27c-53a0-4027-83f6-88c52c758bb1,448d1096-2017-4901-a560-f652a83c737e,OBD_2020_II,3,2024-Q4,Aluminiumblech,2,Alu unbeschichtet od. eloxiert (Sz. Knetleg zu...,RC+,CL+,0.75


### Data Cleaning

In [6]:
# identifying the columns with missing values
missing_values = tbau_df.isnull().sum()

In [7]:
missing_values

oekobaudatProcessUuid          87
oekobaudatDatastockUuid        84
oekobaudatDatastockName        84
productId                       0
tBaustoffVersion                0
productName                     0
eolCategoryId                   0
eolCategoryName                 0
eolScenarioUnbuiltReal          0
eolScenarioUnbuiltPotential     0
technologyFactor                0
dtype: int64

We have 87 products of tbaustoff that are not in the OKD.
Let's see if we can impute them by trying to find the product name in OKD

In [8]:
missing_name = tbau_df[tbau_df["oekobaudatProcessUuid"].isnull()]
missing_name.head()

,oekobaudatProcessUuid,oekobaudatDatastockUuid,oekobaudatDatastockName,productId,tBaustoffVersion,productName,eolCategoryId,eolCategoryName,eolScenarioUnbuiltReal,eolScenarioUnbuiltPotential,technologyFactor
16,NaN,NaN,NaN,5,2024-Q4,Asphalttragschicht - Hochbau,4,Asphalt (Hochbau),Dep+,CL+,0.75
26,NaN,NaN,NaN,7,2024-Q4,"Balkenschichtholz Nadelholz - unbeschichtet, n...",6,"Holz - massiv, naturbelassen",RC+,CL+,0.75
30,NaN,NaN,NaN,39,2024-Q4,Brettsperrholz,6,"Holz - massiv, naturbelassen",RC+,CL+,0.75
31,NaN,NaN,NaN,157,2024-Q4,Konstruktionsvollholz - naturbelassen,6,"Holz - massiv, naturbelassen",RC+,CL+,0.75
56,NaN,NaN,NaN,9,2024-Q4,Beton - Carbonfasern / Textilbeton,8,Beton - Carbonfasern,Dep-,Dep-,0.00


In [9]:
# looking for the missing values in the OBD dataframe
missing_obd = obd_df[obd_df["Name (en)"] == "Konstruktionsvollholz - naturbelassen"]
missing_obd

,UUID,Version,Name (de),Name (en),Kategorie (original),Kategorie (en),Konformität,Laenderkennung,Typ,Referenzjahr,...,HTPnc (A2),IRP (A2),SOP (A2),ODP (A2),POCP (A2),ADPF (A2),ADPE (A2),WDP (A2),source_year,impact_standard


In [10]:
#check duplicated values in tbau_df
duplicated_rows = tbau_df[tbau_df.duplicated()]
print("duplicated rows: ",len(duplicated_rows))

duplicated rows:  0


### Unique Values

In [11]:
obd_uuids = obd_df['UUID'].nunique()
print (obd_uuids, "unique processes in the OBD data")


3863 unique processes in the OBD data


In [12]:
obd_unique_names = obd_df['Name (en)'].nunique()
print(obd_unique_names," unique materials in the OBD data set")


2495  unique materials in the OBD data set


In [13]:
# unique products id 
tbau_df["productId"].nunique()

345

We find different processuuid for same products id. This is due to the fact that the uuid of a product changes in each version/year of the OBD (at least in Tbaustoff)

In [14]:
# example for productid 2 (Aluminium Profil) of tbaustoff   
aluminum_2020 = obd_df[obd_df["UUID"] == "bd6d6d89-b76d-4002-a217-afffbb8aa308"]
aluminum_2020[["Name (en)","source_year","Modul","AP (A2)", "GWPtotal (A2)", "AP", "GWP"]]

,Name (en),source_year,Modul,AP (A2),GWPtotal (A2),AP,GWP
24562,Aluminium section,2020,D,NaN,NaN,-0.025543,-7.889704
24563,Aluminium section,2020,C2,NaN,NaN,0.000005,0.002553
24564,Aluminium section,2020,C1,NaN,NaN,0.000000,0.000000
24565,Aluminium section,2020,A1-A3,NaN,NaN,0.032528,10.677035


In [15]:
aluminum_2023 = obd_df[obd_df["UUID"] == "fdc99ab8-d843-44ec-a66c-92367d244321"]
aluminum_2023[["Name (en)","source_year","Modul","AP (A2)", "GWPtotal (A2)","AP", "GWP"]]

,Name (en),source_year,Modul,AP (A2),GWPtotal (A2),AP,GWP
15297,Aluminium section,2023,A1-A3,0.035951,10.697766,NaN,NaN
15298,Aluminium section,2023,C1,0.000000,0.000000,NaN,NaN
15299,Aluminium section,2023,C2,0.000010,0.003254,NaN,NaN
15300,Aluminium section,2023,D,-0.025415,-7.276583,NaN,NaN


In [16]:
aluminum_2024 = obd_df[obd_df["UUID"] == "3feca796-791b-46d3-8160-95ef243ffb9d"]
aluminum_2024[["Name (en)","source_year","Modul","AP (A2)", "GWPtotal (A2)","AP", "GWP"]]

,Name (en),source_year,Modul,AP (A2),GWPtotal (A2),AP,GWP
2254,Aluminium section,2024,A1-A3,0.040735,10.896927,NaN,NaN
2255,Aluminium section,2024,C2,0.000005,0.004260,NaN,NaN
2256,Aluminium section,2024,C3,0.000000,0.000000,NaN,NaN
2257,Aluminium section,2024,D,-0.029156,-7.511520,NaN,NaN


same product id linked to 3 different UUID

Are the metrics different in different years?
OKD 2020 was working with E 15804 +A1, so is missing all the information of the +A2 columns
There's not much difference between oko 2023 and 2024, at least in the metrics examined, but they are not exactly the same.
Missing values of 2023 or 204 could definitely be imputed with the metrics of the other year. Could have the same approach for year 2020 and previous datasets. 

The total number of actually unique processes in oko is less than expected, given that we can find a same product name with different uuid (depending on the source year). A rough estimate would be to divide the total number of uuids by 3, but not necessarily all processes will be present in all years. 

SUMMARY:

- 345 tbaustoff materials matching with 728 processes of OBD (of which many are same processes but in different version/year). 
- 2619 number of processes in OBD (after removing duplicates for different versions), so missing 1891 to fully match with tbs

In the OBD there are :
- 3863 unique processes (2619 if we consider that some of them are represented with different uuids depending of the version of the dataset)
- 2495  unique materials 

In [17]:
# determining actual unique UUIDS. find rows with same name and modul

# group by name and modul
grouped_df = obd_df.groupby(['Name (en)', 'Modul']).size().reset_index(name='count')

duplicates = grouped_df[grouped_df['count'] > 1]
duplicates

,Name (en),Modul,count
0,1.2.04Expanded clay sand,A1-A3,2
72,3- and 5-layer solid wood panel (German average),A1,2
73,3- and 5-layer solid wood panel (German average),A1-A3,2
74,3- and 5-layer solid wood panel (German average),A2,2
75,3- and 5-layer solid wood panel (German average),A3,2
...,...,...,...
18087,voestalpine rails,D,2
18088,wall elements,A1-A3,2
18089,wall elements,C1,2
18090,wall elements,C2,2


these processes are the actual repeated ones. 

In [18]:
obd_df[obd_df["Name (en)"] == "1.2.04Expanded clay sand"]

,UUID,Version,Name (de),Name (en),Kategorie (original),Kategorie (en),Konformität,Laenderkennung,Typ,Referenzjahr,...,HTPnc (A2),IRP (A2),SOP (A2),ODP (A2),POCP (A2),ADPF (A2),ADPE (A2),WDP (A2),source_year,impact_standard
1495,8ac5659c-5918-41bb-966d-91efdd6c4e50,20.24.070,Blähton Sand,1.2.04Expanded clay sand,'Mineralische Baustoffe' / 'Zuschläge' / 'Bläh...,'Mineral building products' / 'Concrete aggreg...,'EN 15804+A2 (EF 3.1)',DE,generic dataset,2023.0,...,NaN,NaN,NaN,1.870000e-12,0.001075,4.812913,2.124977e-08,0.023960,2024,A2
15973,008280d2-302f-4735-b845-1b3af6f14151,20.23.050,Blähton Sand,1.2.04Expanded clay sand,'Mineralische Baustoffe' / 'Zuschläge' / 'Bläh...,'Mineral building products' / 'Concrete aggreg...,'EN 15804+A2',DE,generic dataset,2022.0,...,NaN,NaN,NaN,1.595500e-12,0.001071,4.911426,1.548417e-08,0.022202,2023,A2


they both correspond to the same material and process (modul A3), yet they have different UUID (because of the source_year/version). The env metrics are almost the same, but enough to drop them as duplicates? In the next step I will ran a trial to see how it would look like

In [19]:
# Sort by source year descending to keep the latest entry in case of duplicates
obd_df_sorted = obd_df.sort_values(by="source_year", ascending=False)

# Drop duplicates keeping the latest version (based on UUID, Version, Modul)
obd_deduped_df = obd_df_sorted.drop_duplicates(subset=["Name (en)", "Modul"], keep="first") 

# Reset index and show result
obd_deduped_df.reset_index(drop=True, inplace=True)

In [20]:
# write as csv
obd_deduped_df.to_csv("obd_merged_deduped.csv", index=False, sep=";")
#NOTE - by removing duplicates, we are losing the information of the different versions of the same material. Right now we are keeping the latest version, but could happen that some of the metrics are only present in the previous versions.

In [21]:
obd_test = obd_df[["Name (en)","source_year","Modul","AP (A2)", "GWPtotal (A2)", "AP", "GWP"]]
# filtering for rows with missing values in GWPtotal (A2) and AP is not null
obd_test[obd_test["GWPtotal (A2)"].isna() & obd_test["AP"].notna() & obd_test["AP"] != 0]

,Name (en),source_year,Modul,AP (A2),GWPtotal (A2),AP,GWP
14423,NaN,2024,C1,NaN,NaN,0.002470,0.608000
14424,NaN,2024,C2,NaN,NaN,0.029200,4.980000
14425,NaN,2024,C3,NaN,NaN,0.013500,1.830000
14426,NaN,2024,C4,NaN,NaN,0.025800,4.070000
14427,NaN,2024,D,NaN,NaN,-0.020500,-6.200000
...,...,...,...,...,...,...,...
25655,QNG Electricity grid mix OBD 2020_II,2020,B6,NaN,NaN,0.000628,0.532030
25660,NaN,2020,C4,NaN,NaN,0.007661,81.304757
25662,BauderTHERMOFOL U / M,2020,A5,NaN,NaN,0.000345,0.242000
25663,NaN,2020,A1-A3,NaN,NaN,0.036600,25.100000


In [22]:
print("Number of rows in OBD :", len(obd_df))
print("Number of rows in OBD_deduped :", len(obd_deduped_df))

Number of rows in OBD : 25665
Number of rows in OBD_deduped : 18118


There are 6741 rows that are missing values for A2 metrics (latest version of dataset), but have it in the previous versions. By removing duplicates I'm loosing this values, so we need to assess if we are going to need them later

In [24]:
obd_deduped_df["UUID"].nunique()

2619

In [25]:
obd_deduped_df["Name (en)"].nunique()

2495

In [26]:
obd_deduped_df[obd_deduped_df["Name (en)"] == "Aluminium section"]

,UUID,Version,Name (de),Name (en),Kategorie (original),Kategorie (en),Konformität,Laenderkennung,Typ,Referenzjahr,...,HTPnc (A2),IRP (A2),SOP (A2),ODP (A2),POCP (A2),ADPF (A2),ADPE (A2),WDP (A2),source_year,impact_standard
2203,3feca796-791b-46d3-8160-95ef243ffb9d,20.24.070,Aluminium Profil,Aluminium section,'Metalle' / 'Aluminium' / 'Aluminiumprofil','Metals' / 'Aluminium' / 'Aluminium profiles','EN 15804+A2 (EF 3.1)',DE,generic dataset,2023.0,...,NaN,NaN,NaN,1.300000e-15,0.000005,0.054593,7.055990e-10,0.000030,2024,A2
2204,3feca796-791b-46d3-8160-95ef243ffb9d,20.24.070,Aluminium Profil,Aluminium section,'Metalle' / 'Aluminium' / 'Aluminiumprofil','Metals' / 'Aluminium' / 'Aluminium profiles','EN 15804+A2 (EF 3.1)',DE,generic dataset,2023.0,...,NaN,NaN,NaN,7.550180e-11,0.027316,131.913039,1.091803e-06,1.610595,2024,A2
2233,3feca796-791b-46d3-8160-95ef243ffb9d,20.24.070,Aluminium Profil,Aluminium section,'Metalle' / 'Aluminium' / 'Aluminiumprofil','Metals' / 'Aluminium' / 'Aluminium profiles','EN 15804+A2 (EF 3.1)',DE,generic dataset,2023.0,...,NaN,NaN,NaN,-4.688030e-11,-0.018711,-87.274090,-6.219902e-07,-1.211407,2024,A2
2234,3feca796-791b-46d3-8160-95ef243ffb9d,20.24.070,Aluminium Profil,Aluminium section,'Metalle' / 'Aluminium' / 'Aluminiumprofil','Metals' / 'Aluminium' / 'Aluminium profiles','EN 15804+A2 (EF 3.1)',DE,generic dataset,2023.0,...,NaN,NaN,NaN,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,2024,A2
14177,fdc99ab8-d843-44ec-a66c-92367d244321,20.23.050,Aluminium Profil,Aluminium section,'Metalle' / 'Aluminium' / 'Aluminiumprofil','Metals' / 'Aluminium' / 'Aluminium profiles','EN 15804+A2',DE,generic dataset,2022.0,...,NaN,NaN,NaN,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,2023,A2


Went from 12 rows to 5, one for each process (instead of nr of processes x number of versions)

In [27]:
obd_deduped_df[obd_deduped_df["Name (en)"] == "Aluminium section"]["Modul"]

2203        C2
2204     A1-A3
2233         D
2234        C3
14177       C1
Name: Modul, dtype: object

In [28]:
obd_df[obd_df["UUID"] == "27f20dc1-5529-4194-8a06-1ae5b7ba6a51"]["Szenario"]

11559              energetisch
11581                      NaN
11582                      NaN
11583                      NaN
11584                      NaN
11585    Entsorgung Verpackung
11586                      NaN
11587                      NaN
Name: Szenario, dtype: object

In [29]:
obd_df[obd_df["Name (en)"] == "Aluminium section"]

,UUID,Version,Name (de),Name (en),Kategorie (original),Kategorie (en),Konformität,Laenderkennung,Typ,Referenzjahr,...,HTPnc (A2),IRP (A2),SOP (A2),ODP (A2),POCP (A2),ADPF (A2),ADPE (A2),WDP (A2),source_year,impact_standard
2254,3feca796-791b-46d3-8160-95ef243ffb9d,20.24.070,Aluminium Profil,Aluminium section,'Metalle' / 'Aluminium' / 'Aluminiumprofil','Metals' / 'Aluminium' / 'Aluminium profiles','EN 15804+A2 (EF 3.1)',DE,generic dataset,2023.0,...,NaN,NaN,NaN,7.550180e-11,0.027316,131.913039,1.091803e-06,1.610595,2024,A2
2255,3feca796-791b-46d3-8160-95ef243ffb9d,20.24.070,Aluminium Profil,Aluminium section,'Metalle' / 'Aluminium' / 'Aluminiumprofil','Metals' / 'Aluminium' / 'Aluminium profiles','EN 15804+A2 (EF 3.1)',DE,generic dataset,2023.0,...,NaN,NaN,NaN,1.300000e-15,0.000005,0.054593,7.055990e-10,0.000030,2024,A2
2256,3feca796-791b-46d3-8160-95ef243ffb9d,20.24.070,Aluminium Profil,Aluminium section,'Metalle' / 'Aluminium' / 'Aluminiumprofil','Metals' / 'Aluminium' / 'Aluminium profiles','EN 15804+A2 (EF 3.1)',DE,generic dataset,2023.0,...,NaN,NaN,NaN,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,2024,A2
2257,3feca796-791b-46d3-8160-95ef243ffb9d,20.24.070,Aluminium Profil,Aluminium section,'Metalle' / 'Aluminium' / 'Aluminiumprofil','Metals' / 'Aluminium' / 'Aluminium profiles','EN 15804+A2 (EF 3.1)',DE,generic dataset,2023.0,...,NaN,NaN,NaN,-4.688030e-11,-0.018711,-87.274090,-6.219902e-07,-1.211407,2024,A2
15297,fdc99ab8-d843-44ec-a66c-92367d244321,20.23.050,Aluminium Profil,Aluminium section,'Metalle' / 'Aluminium' / 'Aluminiumprofil','Metals' / 'Aluminium' / 'Aluminium profiles','EN 15804+A2',DE,generic dataset,2022.0,...,NaN,NaN,NaN,2.064770e-11,0.019711,146.197295,4.722078e-07,0.525069,2023,A2
15298,fdc99ab8-d843-44ec-a66c-92367d244321,20.23.050,Aluminium Profil,Aluminium section,'Metalle' / 'Aluminium' / 'Aluminiumprofil','Metals' / 'Aluminium' / 'Aluminium profiles','EN 15804+A2',DE,generic dataset,2022.0,...,NaN,NaN,NaN,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,2023,A2
15299,fdc99ab8-d843-44ec-a66c-92367d244321,20.23.050,Aluminium Profil,Aluminium section,'Metalle' / 'Aluminium' / 'Aluminiumprofil','Metals' / 'Aluminium' / 'Aluminium profiles','EN 15804+A2',DE,generic dataset,2022.0,...,NaN,NaN,NaN,5.000000e-16,0.000009,0.043828,2.289147e-10,0.000017,2023,A2
15300,fdc99ab8-d843-44ec-a66c-92367d244321,20.23.050,Aluminium Profil,Aluminium section,'Metalle' / 'Aluminium' / 'Aluminiumprofil','Metals' / 'Aluminium' / 'Aluminium profiles','EN 15804+A2',DE,generic dataset,2022.0,...,NaN,NaN,NaN,-4.175700e-12,-0.012814,-97.733143,-2.537972e-07,-0.448489,2023,A2
24562,bd6d6d89-b76d-4002-a217-afffbb8aa308,20.19.120,Aluminium Profil,Aluminium section,'Metalle' / 'Aluminium' / 'Aluminiumprofil','Metals' / 'Aluminium' / 'Aluminium profiles','DIN EN 15804',DE,generic dataset,2018.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020,A1
24563,bd6d6d89-b76d-4002-a217-afffbb8aa308,20.19.120,Aluminium Profil,Aluminium section,'Metalle' / 'Aluminium' / 'Aluminiumprofil','Metals' / 'Aluminium' / 'Aluminium profiles','DIN EN 15804',DE,generic dataset,2018.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020,A1


In [30]:
# rows where name is missing
missing_name = obd_df[(obd_df["Name (en)"].isna()) & (obd_df["Name (de)"].isna())]
print ("Number of rows with missing names in english and german:", len(missing_name))

Number of rows with missing names in english and german: 0


The missing rows for names are just for one of the language columns, so they all have names in one language at least

In [31]:
# unique values in the 'OBD' column
tbau_df["oekobaudatProcessUuid"].nunique()

728

In [32]:
len(tbau_df["oekobaudatProcessUuid"])


815

815 - 728  = 87 
Each uuid is unique (not like in obd). The remaining are missing values
Note that data entries in `ÖKOBAUDAT` are referred to as *Processes*, not materials, as life cycle assessment is conducted at the process level. Since in the tbaustoff dataset we are talking of *ProcessUuid* in connection with `ÖKOBAUDAT`, it means that the data is related to a speficic material combined with assessments through its entire life cycle.

In [33]:
# looking at the missing values
tbau_df[tbau_df["oekobaudatProcessUuid"].isnull()]

,oekobaudatProcessUuid,oekobaudatDatastockUuid,oekobaudatDatastockName,productId,tBaustoffVersion,productName,eolCategoryId,eolCategoryName,eolScenarioUnbuiltReal,eolScenarioUnbuiltPotential,technologyFactor
16,NaN,NaN,NaN,5,2024-Q4,Asphalttragschicht - Hochbau,4,Asphalt (Hochbau),Dep+,CL+,0.75
26,NaN,NaN,NaN,7,2024-Q4,"Balkenschichtholz Nadelholz - unbeschichtet, n...",6,"Holz - massiv, naturbelassen",RC+,CL+,0.75
30,NaN,NaN,NaN,39,2024-Q4,Brettsperrholz,6,"Holz - massiv, naturbelassen",RC+,CL+,0.75
31,NaN,NaN,NaN,157,2024-Q4,Konstruktionsvollholz - naturbelassen,6,"Holz - massiv, naturbelassen",RC+,CL+,0.75
56,NaN,NaN,NaN,9,2024-Q4,Beton - Carbonfasern / Textilbeton,8,Beton - Carbonfasern,Dep-,Dep-,0.00
...,...,...,...,...,...,...,...,...,...,...,...
770,NaN,NaN,NaN,286,2024-Q4,"Splitt 2/8, dauerelastisch gebunden (Latex, so...",124,SchÃ¼ttung organ./min. gebunden,Dep-,Dep-,0.00
771,NaN,22885a6e-1765-4ade-a35e-ae668bd07256,OBD_2023_I,288,2024-Q4,SplittschÃ¼ttung mineral. geb.,125,SchÃ¼ttung min./min. gebunden,RC-,RC-,1.00
772,NaN,ca70a7e6-0ea4-4e90-a947-d44585783626,OBD_2024_I,288,2024-Q4,SplittschÃ¼ttung mineral. geb.,125,SchÃ¼ttung min./min. gebunden,RC-,RC-,1.00
773,NaN,448d1096-2017-4901-a560-f652a83c737e,OBD_2020_II,288,2024-Q4,SplittschÃ¼ttung mineral. geb.,125,SchÃ¼ttung min./min. gebunden,RC-,RC-,1.00


| Code   | Likely Meaning                                  |
| ------ | ----------------------------------------------- |
| `EV+`  | Energy recovery with net benefit                |
| `EV-`  | Energy recovery with net loss (e.g., emissions) |
| `RC+`  | Material recycling with positive circularity    |
| `RC-`  | Recycling with limited value recovery           |
| `CL+`  | Closed-loop recycling (same use)                |
| `CL-`  | Closed-loop not feasible                        |
| `Dep+` | Disposal with minor recoverable output          |
| `Dep-` | Full landfill (no recovery)                     |
| `EB`   | End-burning (e.g. incineration)                 |
| `SV`   | Substitutional value or low-grade reuse         |


#REVIEW - in tbaustoff, do different products with same eolcategory share the same eol scenario and technology factor? My hipothesis is that they do, and that's why we can associate them with similar pollutants.

In [34]:
tbau_df[tbau_df["productName"]=="Asphalttragschicht - Hochbau"]

,oekobaudatProcessUuid,oekobaudatDatastockUuid,oekobaudatDatastockName,productId,tBaustoffVersion,productName,eolCategoryId,eolCategoryName,eolScenarioUnbuiltReal,eolScenarioUnbuiltPotential,technologyFactor
16,NaN,NaN,NaN,5,2024-Q4,Asphalttragschicht - Hochbau,4,Asphalt (Hochbau),Dep+,CL+,0.75


In [35]:
# Removing duplicates of the tbau_df dataframe
# Sort by source year descending to keep the latest entry in case of duplicates
tbs_sorted = tbau_df.sort_values(by="oekobaudatDatastockName", ascending=False)

# Drop duplicates keeping the latest version (based on UUID, Version, Modul)
tbs_deduped = tbs_sorted.drop_duplicates(subset=["productName"], keep="first") 

# Reset index and show result
tbs_deduped.reset_index(drop=True, inplace=True)

In [36]:
tbau_df.shape

(815, 11)

In [37]:
tbs_deduped.shape

(338, 11)

In [38]:
tbs_deduped.to_csv("tbs_deduped.csv", index=False, sep=";")

# Pollutants Dataset

In [39]:
# detect files encoding for pollutant_combinations.csv
with open("/Users/pablosoriano/Documents/Data Science/bbsr-challenge/csv/pollutant_combinations.csv", "rb") as f:
    result = chardet.detect(f.read(100000))  # Read first 100,000 bytes
    print(result)
# importing pullutant_combinations.csv
pollutants_df = pd.read_csv("/Users/pablosoriano/Documents/Data Science/bbsr-challenge/csv/pollutant_combinations.csv", delimiter=",", encoding = result["encoding"], low_memory=False)

{'encoding': 'utf-8', 'confidence': 0.99, 'language': ''}


In [40]:
pollutants_df

,Baumaterial ohne Fremd-/Störstoffe,Fremd-/Störstoffbeschreibung,Störstoffklasse
0,Gussasphaltestrich,ohne Fremd-/Störstoffe,S0
1,Gussasphaltestrich,"Klebstoff-, Bodenbelagsreste, Trennfolien",S2
2,Gussasphaltestrich,Heizungsverteilrohre,S3
3,Beton,ohne Fremd-/Störstoffe,S0
4,Beton,Stahlbewehrung,S2
...,...,...,...
142,"Keramik-, Klinkerplatten, -böden",Kalkmörtel (leicht trennbar),S1
143,"Keramik-, Klinkerplatten, -böden","Kalkzementmörtel, Imprägnierung",S2
144,Glasbaustein,ohne Fremd-/Störstoffe,S0
145,Glasbaustein,Mörtel; Bewehrungsstahl im Mörtel,S2


In [41]:
pollutants_df["Baumaterial ohne Fremd-/Störstoffe"].nunique()

66

In [42]:
pollutants_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147 entries, 0 to 146
Data columns (total 3 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   Baumaterial ohne Fremd-/Störstoffe  147 non-null    object
 1   Fremd-/Störstoffbeschreibung        147 non-null    object
 2   Störstoffklasse                     147 non-null    object
dtypes: object(3)
memory usage: 3.6+ KB


## group pollutant categories 

There are many semantically identical or near-identical pollutant descriptions that vary in phrasing. For instance:

All the following under S0 are likely the same:

"ohne Fremd-/Störstoffe"

"ohne Fremd-/Störstoffe (nicht beschichtet, nicht kaschiert)"

"ohne Fremd-/Störstoffe (unbeschichtet oder eloxiert)"

#TODO - predict pollutant class instead of pollutant itself

In [43]:
import pandas as pd

# Function to correct similar pollutant descriptions
def canonicalize_description(desc):
    if pd.isna(desc):
        return desc
    desc = desc.strip().lower()
    
    if desc.startswith("feinkörn. material"):
        return "feinkörn. Material (Putze, Mörtel); org. Verbind., Bitumenreste, Gips in geringen Mengen (z.B. Feuchteabdichtungen, Spachteln, Anstriche)"
    elif desc.startswith("geringfügig verunreinigt"):
        return "geringfügig verunreinigt (konv. Besch, Klebstoffreste, Belagsreste, Bitumenreste)"
    elif desc.startswith("ohne fremd-/"):
        return "ohne Fremd-/Störstoffe"
    elif desc.startswith("org. v"):
        return "org. V., Bitumen, Gips, Leichtanteile in größeren Mengen (z.B. Gipsputz, Bitumendickschicht, Folien, Flüssigabdichtungen, Dämmstoffreste); Epoxidharz in geringen Mengen (Fugenmassen bei WU Beton, Feuchteabdichtung)"
    elif desc.startswith("heizungsverteilrohre"):
        return "Heizungsverteilrohre; verunreinigt mit Dämmstoff, Kunststoff/Bitumen (Bahnen, Beschichtungen, Flüssigfolie/-abdichtung)"
    elif desc.startswith("klebstof"):
        return "Klebstoff-, Bodenbelagsreste, Trennfolien"
    elif desc in ["lehmfarbe", "lehmputz, lehmfarbe"]:
        return "Lehmputz, Lehmfarbe"
    elif desc.startswith("putze, klebespachtel"):
        return "Putze, Klebespachtel (WDVS), Alukaschierung, Gipskarton"
    elif desc.startswith("verunreinigungen mit natürl."):
        return "Verunreinigungen mit natürl. Materialien (Naturfarbe, Kalkmörtel, Armierungsgewebe)"
    
    return desc  # fallback to unchanged if no match

# Apply the rule-based mapping
pollutants_df["Fremd-/Störstoffbeschreibung"] = pollutants_df["Fremd-/Störstoffbeschreibung"].apply(canonicalize_description)




In [44]:
# Check for unique values in the 'Fremd-/Störstoffbeschreibung' column
unique_values = pollutants_df["Fremd-/Störstoffbeschreibung"].nunique()
unique_values

37

In [46]:
# Merge pollutants_df with tbau_df on productName and Baumaterial ohne Fremd-/Störstoffe
merged_df = tbs_deduped.merge(
    pollutants_df,
    left_on="productName",
    right_on="Baumaterial ohne Fremd-/Störstoffe",
    how="inner"
)

# Display the merged dataframe
merged_df

,oekobaudatProcessUuid,oekobaudatDatastockUuid,oekobaudatDatastockName,productId,tBaustoffVersion,productName,eolCategoryId,eolCategoryName,eolScenarioUnbuiltReal,eolScenarioUnbuiltPotential,technologyFactor,Baumaterial ohne Fremd-/Störstoffe,Fremd-/Störstoffbeschreibung,Störstoffklasse
0,c7381cc7-53b7-427e-9c0b-c9edc4152602,ca70a7e6-0ea4-4e90-a947-d44585783626,OBD_2024_I,108,2024-Q4,Gussasphaltestrich,59,Gussasphaltestrich,Dep+,CL+,0.75,Gussasphaltestrich,ohne Fremd-/Störstoffe,S0
1,c7381cc7-53b7-427e-9c0b-c9edc4152602,ca70a7e6-0ea4-4e90-a947-d44585783626,OBD_2024_I,108,2024-Q4,Gussasphaltestrich,59,Gussasphaltestrich,Dep+,CL+,0.75,Gussasphaltestrich,"Klebstoff-, Bodenbelagsreste, Trennfolien",S2
2,c7381cc7-53b7-427e-9c0b-c9edc4152602,ca70a7e6-0ea4-4e90-a947-d44585783626,OBD_2024_I,108,2024-Q4,Gussasphaltestrich,59,Gussasphaltestrich,Dep+,CL+,0.75,Gussasphaltestrich,Heizungsverteilrohre; verunreinigt mit Dämmsto...,S3
3,fdbe82c6-7073-4f8c-a540-c37a6bf16cd6,ca70a7e6-0ea4-4e90-a947-d44585783626,OBD_2024_I,68,2024-Q4,Expandierter Kork,39,Kork,EV-,RC+,0.75,Expandierter Kork,"putz, klebespachtel, klebstoff",S3
4,86d919ee-8f30-4ca4-9b7e-717aecba6ac0,ca70a7e6-0ea4-4e90-a947-d44585783626,OBD_2024_I,342,2024-Q4,Zementestrich,138,Estrich CT - Zementestrich,Dep+,RC-,0.25,Zementestrich,ohne Fremd-/Störstoffe,S0
5,86d919ee-8f30-4ca4-9b7e-717aecba6ac0,ca70a7e6-0ea4-4e90-a947-d44585783626,OBD_2024_I,342,2024-Q4,Zementestrich,138,Estrich CT - Zementestrich,Dep+,RC-,0.25,Zementestrich,"geringfügig verunreinigt (konv. Besch, Klebsto...",S2
6,86d919ee-8f30-4ca4-9b7e-717aecba6ac0,ca70a7e6-0ea4-4e90-a947-d44585783626,OBD_2024_I,342,2024-Q4,Zementestrich,138,Estrich CT - Zementestrich,Dep+,RC-,0.25,Zementestrich,Heizungsverteilrohre; verunreinigt mit Dämmsto...,S3
7,NaN,NaN,NaN,182,2024-Q4,Lehmbauplatte,90,Lehmbauplatte,SV,CL+,0.75,Lehmbauplatte,"Lehmputz, Lehmfarbe",S1
8,NaN,NaN,NaN,182,2024-Q4,Lehmbauplatte,90,Lehmbauplatte,SV,CL+,0.75,Lehmbauplatte,Verunreinigungen mit natürl. Materialien (Natu...,S2
9,NaN,NaN,NaN,182,2024-Q4,Lehmbauplatte,90,Lehmbauplatte,SV,CL+,0.75,Lehmbauplatte,"konvent. wandfarbe; zement, kalk, gips, kunsts...",S3
